In [1]:
!pip install ipywidgets --quiet

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider, fixed

In [3]:
def evolve(y0:np.ndarray, t0:float, dt:float,
           n:int, alpha:float, f:callable, method:callable, scale=None)->np.ndarray:
    dof=len(y0)
    t=t0
    a = []
    y1=y0.copy()
    if scale is None:
        for i in range(1,n):
            a.append(y1)
            y1=method(y1, t, alpha, dt, f)
            t+=dt
    else:
        if len(scale) != dof+1:
            raise Exception(f'scale vector must have dim={dof+1}')
        t_scale=scale[0]
        dt=dt/t_scale
        scal = np.array(scale[1:])
        y=y1/scal
        for i in range(1,n):
            a.append(y1)
            y=method(y, t, alpha, dt, f)
            y1=y*scal
            t+=dt
    return np.array(a)

def euler_step(y:np.ndarray, t:float, alpha:float, dt:float, f:callable)->np.ndarray:
    y =y + dt * f(y,t,alpha) # Note that + and * are defined operations on n
    return y


# Verlet step assumes that the xi stores quantities as
# x1,v1,x2,v2 ...
def verlet_step(y:np.ndarray, t:float, alpha:float, dt:float,
                f:callable)->np.ndarray:
    dof=len(y)//2
    for i in range(dof):
        y[2*i+1] = y[2*i+1] + dt*f(y,t,alpha)[2*i+1]
        y[2*i] = y[2*i] + dt*f(y,t,alpha)[2*1]
    return y

# Leap-frog step assumes that xi stores quantities as 
# x1,v1,x2,v2 ...
def leap_frog_step(y:np.ndarray, t:float, alpha:float, dt:float,
                   f:callable)->np.ndarray:
    dof=len(y)//2
    for i in range(dof):
        y[2*i+1] = y[2*i+1] + dt*f(y,t,alpha)[2*i+1]
        y[2*i] = y[2*i] + dt*f(y,t,alpha)[2*1]
    return y

def rk4_step(y:np.ndarray, t:float, alpha:float,
             dt:float,
             f:callable)->np.ndarray:
    k1 = f(y,t,alpha)
    k2 = f(y + k1*dt/2.0, t+dt/2, alpha)
    k3 = f(y + k2*dt/2.0, t+dt/2.0, alpha)
    k4 = f(y + k3*dt,t+dt, alpha)
    k = dt * (k1 + 2*k2 + 2*k3 + k4)/6.0
    y=y+k
    return y


$Define Scaling factors: \\ x=\chi x',\ y=\chi y',\ t=\tau t',\ \chi = a = \dfrac{r_p}{1-e},\ \tau = \sqrt{\dfrac{4\pi^2\chi^3}{G(m_1+m_2)}},\ v_0=\sqrt{\dfrac{(1+e)Gm_1}{r_p}}\ \\
the\ initial/starting\ point\ of\ the\ trajectory\ x_o = \left(\dfrac{r_p}{\chi},0,0,\dfrac{\nu_o\tau}{\chi}\right)\\
\dfrac{d}{dt'}\begin{bmatrix}x'\\ y'\\ v_x'\\ v_y'\end{bmatrix} = \begin{bmatrix}v_x'\\v_y'\\\dfrac{4\pi^2x'}{(x'^2+y'^2)^{3/2}} + \dfrac{\alpha x'}{(x'^2+y'^2)^2}\\\dfrac{4\pi^2y'}{(x'^2+y'^2)^{3/2}} + \dfrac{\alpha y'}{(x'^2+y'^2)^2}\end{bmatrix}\\
\ \ \ \ \ \ \ \ \ \xi \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \zeta \\
\dfrac{d\xi}{dt'}=\zeta $

Process of Fourth Order Approximation of Derivative for Each Function

1. We start with equating k1 to the slope (derivative) of the function at the beginning of the time step such that t = 0.
2. Then, approximate the value for the function at half the time step ($t = dt/2$) using Taylor expansion with the derivative now being the estimated slope at the midpoint of k1 which is then assigned to k2. 
3. We continue with a similar method where we approximate the function at half the time step but the derivative now being the slope at the midpoint of k2 which is then assigned this value to k3.
4. The last step is to approximate the function at the time step $t = dt$ and assign the slope estimation at the end of the time step to be k4.
5. Finally, we approximate the function at the later time step with again the Taylor expansion about $t = 0$ but the first derivative is not the weighted sum of these slopes to get best estimate of derivative at $t = t + dt$.



In [4]:
def orbit(M1, m2, rp, e, N, dt, method, G_SI, G, AU, Year, alpha):
    def zeta(xi, t=None, alpha=0.0, param=None):
        r=(xi[0]*xi[0]+xi[1]*xi[1])**1.5
        relative = alpha/(xi[0]**2 + xi[1]**2)**2
        return np.array([xi[2],xi[3],
                         -4*np.pi*np.pi*xi[0]/r + relative*xi[0],
                         -4*np.pi*np.pi*xi[1]/r + relative*xi[1]])
    
    a= rp/(1-e)
    chi=a
    func = method
    M=M1+m2
    tau=np.sqrt(4*np.pi*np.pi*(a**3)/(G_SI*M)) #temporal scaling parameter (orbital period)
    v = np.sqrt((1+e)*G_SI*M1/rp)
    t_init=0.0
    
    scale=np.array([tau,chi,chi,chi/tau,chi/tau])
    init = np.array([rp,0,0,v])
    t=np.arange(0,N*dt,dt)
    
    traj = evolve(init,t_init,dt,N,alpha, 
                 zeta,func,scale)
    complete_traj = []
    
    for f in traj:
        l=[m2/M*f[0]/AU,m2/M*f[1]/AU,
                    -M1/M*f[0]/AU,-M1/M*f[1]/AU,
                    m2/M*f[2],m2/M*f[3],-M1/M*f[2], -M1/M*f[3]]
        complete_traj.append(l)
    
    return t, complete_traj
            

In [5]:
def estimate_orbit(alpha, methods:list=['rk4_step']):
    
    G_SI=6.67E-11   # Universal gravitation m^3/Kg/s^2
    G=1.9935E-44  #AU^3/Kg/s^2

    M_sun=1.989E30 # Kg
    AU=1.496E11  # m
    Year=31557600  # s

    M_mercury=0.33E24 # Kg
    rp_mercury=46.0E9  # m
    e_mercury=0.205
#Time Step and N = # of Steps
    dt=0.01
    dt=dt*Year
    N=500
        
    # obtaining trajectory of mercury with relativistic correction at alpha =0.2
    t_mercury_w, traj_mercury_w = orbit(M_sun, M_mercury, rp_mercury, e_mercury, N, dt, eval(methods),
                                      G_SI, G, AU, Year, alpha)
    x_merc_w=[frame[2] for frame in traj_mercury_w]
    y_merc_w=[frame[3] for frame in traj_mercury_w]


    # obtaining trajectory of mercury without relativistic correction
    t_mecury, traj_mercury = orbit(M_sun, M_mercury, rp_mercury, e_mercury, N, dt, eval(methods),
                                  G_SI, G, AU, Year, alpha=0.0)
    
    x_merc=[frame[2] for frame in traj_mercury]
    y_merc=[frame[3] for frame in traj_mercury]

    fig, axs = plt.subplots(figsize=(8,8))
    font = {'family': 'serif', 'color': 'black', 'weight': 'bold', 'size': 16}

    cir = plt.Circle((0,0),0.015, color = 'goldenrod', label = 'Sun = Center of Mass')
    axs.add_patch(cir)
    axs.plot(x_merc_w, y_merc_w, 'r-', label='Corrected')
    axs.plot(x_merc, y_merc, 'k-', label='Uncorrected')
    axs.set_xlabel('$X_{AU}$', fontdict=font, labelpad=10)
    axs.set_ylabel('$Y_{AU}$', fontdict=font, labelpad=10)

    axs.legend(loc = 'upper right')
    plt.show()

In [6]:
interact(estimate_orbit, alpha=widgets.FloatSlider(min=-0.0, max=2.0, step=0.01, value=0.2),
         methods=["rk4_step", "euler_step", "verlet_step", "leap_frog_step"]);


interactive(children=(FloatSlider(value=0.2, description='alpha', max=2.0, step=0.01), Dropdown(description='m…

In [7]:
interact(estimate_orbit, alpha=widgets.FloatSlider(min=-0.0, max=2.0, step=0.01, value=0.5),
         methods=["rk4_step", "euler_step", "verlet_step", "leap_frog_step"]);

interactive(children=(FloatSlider(value=0.5, description='alpha', max=2.0, step=0.01), Dropdown(description='m…

In [8]:
interact(estimate_orbit, alpha=widgets.FloatSlider(min=-0.0, max=2.0, step=0.01, value=1.0),
         methods=["rk4_step", "euler_step", "verlet_step", "leap_frog_step"]);

interactive(children=(FloatSlider(value=1.0, description='alpha', max=2.0, step=0.01), Dropdown(description='m…

### Comments

Below, I provided an interactive widget for evaluating multiple combinations of alpha and integration methods. 
This will help provide a much better comparison between the three methods used.

c) Based on the above plots, as $\alpha$ increases, the precession of Mercury's orbit increases as well as the eccentricity causing the orbits to take a more elongated shape. This would also cause the aphelion (further distance in orbit from Sun) to increase. Therefore, we conclude that the relativistic correction term changes the orientation and eccentricity of a planet's orbit.

d) In RK4, we approximate the function at much smaller times steps since we do higher order approximations of the slope rather than with the Euler method where we only approximate the slope at each time step. Therefore, the orbit of Mercury appears unstable because it will take many and very small time steps to reach accuracy of RK4. 

If we change the time step to 0.001 and increase the number of steps to 1,000, we will notice that the orbit of the planet eventually becomes unstable and starts to drift away from the Sun.